Code based on this [tutorial](https://keras.io/examples/graph/gnn_citations/).

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## Simple MPNN with two layers

### This is how the full MPNN classifier looks.

In [16]:
class MPNN(tf.keras.Model):
    def __init__(
        self,
        node_features,
        edges,
        num_classes,
        hidden_layers = [32,32],
        dropout_rate=0.2,
        normalize=True,
        *args,
        **kwargs,
    ):
        
        super(MPNN, self).__init__(*args, **kwargs)

# In this code we store full graph in memory 

        self.node_features = node_features
        self.edges = edges

# two MPNN layers

        self.layer1 = MPNNlayer(
            hidden_layers,
            dropout_rate,
            name="layer1",
        )
        # Create the second GraphConv layer.
        self.layer2 = MPNNlayer(
            hidden_layers,
            dropout_rate,
            name="layer2",
        )
        
        # Compute logits layer for the classifier (the decoder part)
        self.clas = layers.Dense(units=num_classes, name="logits")

        
    def call(self, batch_indices):
         
        messages_1 = self.layer1(self.node_features,self.edges)
        
        messages_2 = self.layer2(messages_1,self.edges)
        

        ### Now we gather the embeddings in batch_indexes, so that we classify and 
        ### compare the result only for those nodes.
        
        batch_node_embeddings = tf.gather(messages_2, batch_indices)

        # Readout to get the paper subjects from the embeddings
        
        return self.clas(batch_node_embeddings)
        

### Taking a deeper look: first defining our multi-layered perceptrons, then MPNN layers

In [3]:
def create_MLP(hidden_layers, dropout_rate, name=None):
    mlp = []

    for layer in hidden_layers:
        mlp.append(layers.BatchNormalization())
        mlp.append(layers.Dropout(dropout_rate))
        mlp.append(layers.Dense(layer, activation=tf.nn.gelu))

    return keras.Sequential(mlp, name=name)


### MPNN layer.

In [13]:
class MPNNlayer(layers.Layer):
    def __init__(
        self,
        hidden_layers = [32,32],
        dropout_rate=0.2,
        normalize=False,
        *args,
        **kwargs,
    ):
        super(MPNNlayer, self).__init__(*args, **kwargs)
            
        ### These are the two trainable parts of the network. 
        
        self.mlp_aggregate = create_MLP(hidden_layers, dropout_rate)

        self.mlp_update = create_MLP(hidden_layers, dropout_rate)

    def call(self, node_repesentations, edges):

        # node_indexes: for each edge, the source node of the edge
        # neihbor_indexes: for each edge, the target node of the edge
        node_indexes, neighbor_indexes = edges[0], edges[1]
        
        # neighbour_repesentations: for each edge, the embedding of the target node
        neighbour_repesentations = tf.gather(node_repesentations, neighbor_indexes)

        ### Now comes computation:
        
        #Aggregate
        aggregated_messages = self.aggregate(
            node_indexes, neighbour_repesentations, node_repesentations
        )
        
        #Update
        return self.update(node_repesentations, aggregated_messages)

    def aggregate(self, node_indexes, neighbour_messages, node_repesentations):
        num_nodes = node_repesentations.shape[0]
        
        ### Run node features through an MLP
        
        preprocessed_messages = self.mlp_aggregate(neighbour_messages)
        
        ### The sum is taken care of with tf's unsorted segment sum. 
        ### First, get all positions of edges where node i is the source. 
        ### Then, sum all preprocessed messages in this position (they 
        ### have the embedding of the target of the edge). 
        ### The sum is put on the i-th entry of aggregated_message

        aggregated_message = tf.math.unsorted_segment_sum(
            preprocessed_messages, node_indexes, num_segments=num_nodes
        )
        
        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):
        
        ### concatenate aggregated message with my own representation
        updated_messages = tf.concat([node_repesentations, aggregated_messages], axis=1)
        
        ### apply trainable MLP to the result
        
        node_embeddings = self.mlp_update(updated_messages)

        return node_embeddings


## Trying out our model

In [5]:
### Loading Quora dataset
citations = pd.read_csv("cora.cites",
    sep="\t",
    header=None,
    names=["target", "source"],
)

column_names = ["paper_id"] + [f"term_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv("cora.content", sep="\t", header=None, names=column_names,
)

In [6]:
### Some structuring, cleaning

class_values = sorted(papers["subject"].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
citations["source"] = citations["source"].apply(lambda name: paper_idx[name])
citations["target"] = citations["target"].apply(lambda name: paper_idx[name])
papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])

train_data, test_data = [], []

for _, group_data in papers.groupby("subject"):
    # Select around 50% of the dataset for training.
    random_selection = np.random.rand(len(group_data.index)) <= 0.5
    train_data.append(group_data[random_selection])
    test_data.append(group_data[~random_selection])

train_data = pd.concat(train_data).sample(frac=1)
test_data = pd.concat(test_data).sample(frac=1)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)



Train data shape: (1364, 1435)
Test data shape: (1344, 1435)


In [7]:
train_data

,paper_id,term_0,term_1,term_2,term_3,term_4,term_5,term_6,term_7,term_8,...,term_1424,term_1425,term_1426,term_1427,term_1428,term_1429,term_1430,term_1431,term_1432,subject
1936,1590,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2675,593,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2140,2406,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3
2308,2573,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2560,202,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,1413,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
629,418,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2152,1613,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
704,1059,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [8]:
feature_names = set(papers.columns) - {"paper_id", "subject"}
num_features = len(feature_names)
num_classes = len(class_idx)

# Create train and test features as a numpy array.
x_train = train_data["paper_id"].to_numpy()
x_test = test_data["paper_id"].to_numpy()
# Create train and test targets as a numpy array.
y_train = train_data["subject"].to_numpy()
y_test = test_data["subject"].to_numpy()

x_train, y_train

(array([1590,  593, 2406, ..., 1613, 1059,   69]),
 array([1, 6, 3, ..., 1, 3, 0]))

In [17]:
#Adyacency list 
edges = citations[["source", "target"]].to_numpy().T

# Tensorflow vector for each node
node_features = tf.cast(
    papers.sort_values("paper_id")[feature_names].to_numpy(), dtype=tf.dtypes.float32
)

print("Edges shape:", edges.shape)
print("Nodes shape:", node_features.shape)

# Create the GNN. Note how the entire graph is passed onto the model. 

GNN = MPNN(
    node_features = node_features,
    edges = edges,
    num_classes=7,
    hidden_layers=[32,32],
    dropout_rate=0.2,
    name="mpnn_model",
)





Edges shape: (2, 5429)
Nodes shape: (2708, 1433)


/var/folders/gf/12lsjk9d5mn0gdyhlk_xd0zm0000gn/T/ipykernel_73683/3546402807.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  papers.sort_values("paper_id")[feature_names].to_numpy(), dtype=tf.dtypes.float32


In [18]:
GNN.compile(
        optimizer=keras.optimizers.Adam(0.01),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )

# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )


    # Fit the model.
history = GNN.fit(
        x=x_train,
        y=y_train,
        epochs=300,
        batch_size=256,
        validation_split=0.15,
        callbacks=[early_stopping],
    )



Epoch 1/300
5/5 [==============================] - 2s 140ms/step - loss: 2.0577 - acc: 0.2312 - val_loss: 1.9116 - val_acc: 0.1756
Epoch 2/300
5/5 [==============================] - 0s 94ms/step - loss: 1.6502 - acc: 0.3701 - val_loss: 1.8202 - val_acc: 0.3024
Epoch 3/300
5/5 [==============================] - 0s 86ms/step - loss: 1.3990 - acc: 0.4694 - val_loss: 1.6803 - val_acc: 0.3659
Epoch 4/300
5/5 [==============================] - 0s 81ms/step - loss: 1.1751 - acc: 0.5643 - val_loss: 1.9382 - val_acc: 0.3463
Epoch 5/300
5/5 [==============================] - 0s 87ms/step - loss: 0.9449 - acc: 0.6704 - val_loss: 2.6833 - val_acc: 0.3707
Epoch 6/300
5/5 [==============================] - 0s 81ms/step - loss: 0.7612 - acc: 0.7403 - val_loss: 2.7967 - val_acc: 0.3951
Epoch 7/300
5/5 [==============================] - 0s 89ms/step - loss: 0.5820 - acc: 0.7990 - val_loss: 2.9463 - val_acc: 0.4049
Epoch 8/300
5/5 [==============================] - 0s 87ms/step - loss: 0.4242 - acc: 0.8

5/5 [==============================] - 0s 101ms/step - loss: 0.0287 - acc: 0.9871 - val_loss: 0.9615 - val_acc: 0.8293
Epoch 64/300
5/5 [==============================] - 0s 95ms/step - loss: 0.0425 - acc: 0.9888 - val_loss: 0.9445 - val_acc: 0.8244
Epoch 65/300
5/5 [==============================] - 0s 96ms/step - loss: 0.0155 - acc: 0.9957 - val_loss: 0.9260 - val_acc: 0.8341
Epoch 66/300
5/5 [==============================] - 0s 92ms/step - loss: 0.0313 - acc: 0.9888 - val_loss: 0.9770 - val_acc: 0.8244
Epoch 67/300
5/5 [==============================] - 0s 96ms/step - loss: 0.0347 - acc: 0.9914 - val_loss: 0.9831 - val_acc: 0.8195
Epoch 68/300
5/5 [==============================] - 0s 92ms/step - loss: 0.0322 - acc: 0.9888 - val_loss: 1.0130 - val_acc: 0.8195
Epoch 69/300
5/5 [==============================] - 1s 117ms/step - loss: 0.0335 - acc: 0.9888 - val_loss: 1.2117 - val_acc: 0.7902
Epoch 70/300
5/5 [==============================] - 0s 95ms/step - loss: 0.0175 - acc: 0.9965 

In [19]:
GNN.evaluate(x=x_test, y=y_test)

42/42 [==============================] - 1s 16ms/step - loss: 1.1346 - acc: 0.8266


[1.1346373558044434, 0.8266369104385376]

Not bad!